## Read in Rahul's file

In [1]:
import pickle
with open('coord_list.pkl', 'rb') as f:
    data = pickle.load(f)

## Import a few packages 

In [2]:
from astroquery.simbad import Simbad
from astroquery.xmatch import XMatch
from astropy import units as u
from astropy.table import Table
from astropy.coordinates import SkyCoord

## Bring data in my preferred format

In [4]:
ra = []
dec = []
for coord in data:
    ra.append(coord.ra.value)
    dec.append(coord.dec.value)
tab = Table([ra, dec], names=('RA', 'Dec'))
tab.write('test.csv', overwrite=True)

## Use the XMatch service

In [5]:
xmatch_tab = XMatch.query(cat1=open('test.csv'), 
                          cat2='SIMBAD', max_distance=0.5* u.arcsec,
                          colRA1='RA', colDec1='Dec')

In [6]:
orig_coord = SkyCoord(ra=tab['RA']*u.degree, dec=tab['Dec']*u.degree)
xmatch_coord = SkyCoord(ra=xmatch_tab['RA']*u.degree, dec=xmatch_tab['Dec']*u.degree)
idx, d2d, d3d = orig_coord.match_to_catalog_sky(xmatch_coord)

In [7]:
ind = d2d < 0.5 * u.arcsec

In [8]:
print(orig_coord[~ind])

<SkyCoord (ICRS): (ra, dec) in deg
    [(88.69424375,  -6.47762847), (81.57637443,  -6.32723414),
     (85.51903637,  -6.40803371), ..., (80.45860846, -16.77345201),
     (87.9936909 , -17.75482342), (81.2494604 , -17.44843738)]>


## Use Simbad directly

In [9]:
orig_coord = SkyCoord(ra=tab['RA']*u.degree, dec=tab['Dec']*u.degree)
search_radius = [2 * u.arcsec] * len(orig_coord)

In [10]:
result_table = Simbad.query_region(orig_coord,
                                   radius=search_radius)

/usr/local/lib/python3.6/dist-packages/astroquery/simbad/core.py:138: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): '5:54:46.6185 -6:28:39.4625': No astronomical object found :  
  (error.line, error.msg))
/usr/local/lib/python3.6/dist-packages/astroquery/simbad/core.py:138: UserWarning: Warning: The script line number 6 raised an error (recorded in the `errors` attribute of the result table): '6:00:43.2001 -6:30:46.0154': No astronomical object found :  
  (error.line, error.msg))
/usr/local/lib/python3.6/dist-packages/astroquery/simbad/core.py:138: UserWarning: Warning: The script line number 9 raised an error (recorded in the `errors` attribute of the result table): '5:46:19.8309 -6:25:53.8313': No astronomical object found :  
  (error.line, error.msg))
/usr/local/lib/python3.6/dist-packages/astroquery/simbad/core.py:138: UserWarning: Warning: The script line number 12 raised an error (recorded in the `error

In [11]:
result_table.errors

[SimbadError(line=3, msg="'5:54:46.6185 -6:28:39.4625': No astronomical object found :  "),
 SimbadError(line=6, msg="'6:00:43.2001 -6:30:46.0154': No astronomical object found :  "),
 SimbadError(line=9, msg="'5:46:19.8309 -6:25:53.8313': No astronomical object found :  "),
 SimbadError(line=12, msg="'6:03:46.09 -6:31:44.0361': No astronomical object found :  "),
 SimbadError(line=15, msg="'5:44:24.0825 -6:25:19.1065': No astronomical object found :  "),
 SimbadError(line=23, msg="'5:44:42.9476 -6:25:26.0638': No astronomical object found :  "),
 SimbadError(line=32, msg="'6:00:22.2388 -6:33:23.872': No astronomical object found :  "),
 SimbadError(line=45, msg="'5:41:18.6673 -6:30:07.2495': No astronomical object found :  "),
 SimbadError(line=55, msg="'5:41:19.3054 -6:32:59.9194': No astronomical object found :  "),
 SimbadError(line=65, msg="'5:40:55.6155 -6:38:47.3044': No astronomical object found :  "),
 SimbadError(line=73, msg="'5:46:12.6611 -6:42:57.5814': No astronomical obj